In [32]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

In [33]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [34]:
print(f'Number of trainig examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of trainig examples: 25000
Number of testing examples: 25000


In [35]:
print(vars(train_data.examples[0]))

{'text': ['From', 'a', 'modern', 'sensibility', ',', 'it', "'s", 'sometimes', 'hard', 'to', 'watch', 'older', 'films', '.', 'It', "'s", 'annoying', 'to', 'have', 'to', 'watch', 'the', 'stereotypical', 'wallflower', 'librarian', 'have', 'to', 'take', 'off', 'her', 'glasses', 'and', 'become', 'pretty', 'and', 'stupid', 'to', 'win', 'a', 'man', '.', 'Especially', 'such', 'a', 'shallow', 'and', 'inconstant', 'man', '.', 'He', "'s", 'obviously', 'a', 'player', '(', 'I', 'would', "n't", 'trust', 'him', 'to', 'stay', 'true', 'to', 'her', ')', 'who', 'does', "n't", 'want', 'to', 'settle', 'down', ',', 'who', 'only', 'looks', 'at', 'dumb', 'attractive', 'women', 'and', 'always', 'calls', 'them', '"', 'baby', '"', '(', 'ick', '!', ')', '.', 'Even', 'after', 'she', 'totally', 'changes', 'her', 'appearance', 'and', 'her', 'life', 'for', 'him', ',', 'he', 'only', 'goes', 'to', 'her', 'after', 'he', "'s", '(', 'supposedly', ')', 'rejected', 'by', 'another', 'woman', 'and', 'learns', 'that', 'Connie'

In [36]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [37]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [38]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [43]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [44]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 201490), (',', 191070), ('.', 164685), ('and', 109019), ('a', 108721), ('of', 100582), ('to', 93295), ('is', 76021), ('in', 61323), ('I', 53792), ('it', 53054), ('that', 49008), ('"', 43992), ("'s", 43023), ('this', 42179), ('-', 36787), ('/><br', 35349), ('was', 34977), ('as', 30340), ('with', 29854)]


In [45]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [46]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [47]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

## Build the Model

__init__ 안에 모듈의 레이어를 정의한다. 세 개의 층은 embedding layer,
RNN, 그리고 a linear layer이다. 모든 레이어들에는 특별히 정의하지 않은 한
난수값으로 설정된 파라미터들이 있다. 임베딩 레이어는 sparse한 one-hot vector를 dense한 임베딩 벡터로 변환하는데 사용된다. 이 임베딩 레이어는 single fully connected layer이다. RNN의 input 차원을 줄일 뿐만 아니라 리뷰에서 비슷한 감성을 가지고있다고 판단되는 단어들의 theory를 dense한 벡터공간에 매핑시킨다. 
RNN 레이어에서는 one-hot벡터가 임베딩된 dense한 벡터를 가지고 이전의 ht-1에서 다음 hidden state인 ht를 계산하는데 RNN을 사용한다.
마지막으로 linear layer가 hidden state를 계산하고 fully connected 된
$f(h_T)$,를 통해서 ouput dimention으로 변환이된다.(?)


forward method란 이런 모델에 샘플을 투입하는 방식이다.
각각의 배치인 text는 텐서의 사이즈 [sentence length, batch size]이다.
이것은 내부의 각각 단어들이 one-hot벡터로 변환된 문장들의 배치이다.
원핫벡터들때문에 텐서는 다른차원이 필요하지만 pytorch는 간편하게 원핫벡터를 인덱스 값으로 저장한다 즉, 한 문장을 표현하는 텐서는 그 문장 안의 각각의 토큰들의 인덱스들의 텐서와 같다. 토큰들의 리스트를 인덱스의 리스트로 변환하는 작업을 numericalizing이라고 한다. 
인풋 배치는 임베딩 레이어를 통해서 임베딩 되고, 이게 문장들의 dense한 벡터를 나타내게된다. 임베딩된건 텐서의 사이즈 [sentence length, batch size, embedding dim]이다.

이제 임베딩 된 것들이 RNN을 통해 들어가게 된다. 몇몇 프레임워크들에서는 초기 히든 레이어를 직접 RNN에 넣어야 하지만 파이토치에서는 만약 초기 히든레이어의 설정값이 주어지지 않는다면 모든 텐서들을 0으로 간주한다.

RNN은 [sentence length,batch size, hidden dim]의 output size 와 hidden of size인 [1, batch size, hidden dim]의 2개의 텐서를 반환한다.
output은 모든 연속된 시간의 hidden state의 연쇄인 반면 hidden은 단지 마지막 hidden state이다. 우리는 이를 assert statement를 통해서 증명한다. 사이즈 하나의 차원 하나를 줄이기 위해서 squeeze 방법을 쓰는 것을 주목하라.
마지막으로 마지막의 hidden state인 hidden을 선형 레이어 fc를 통과시켜 예측값을 만든다.


In [83]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

이제 RNN 클래스의 인스턴스가 만들어졌다.
입력 차원은 원핫 벡터의 차원들과 같고 이는 단어의 사이즈와 같다.
임베딩 차원은 dense한 단어들의 벡터의 사이즈와 같다. 이는 보통 50-250 개의 차원들이나
보통 단어의 사이즈에 따라서 다르다.
hidden dimention은 hidden states의 차원과 같다. 보통 100에서 500개의 차원들이나 
단어의 사이즈나 밀집된 벡터의 크기나 일의 복잡성에 따라 달라진다.



In [84]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

이제 이 모델이 얼마나 많은 학습가능한 매개변수들을 가지고 있는지 알려주는 함수를 만들어 다른모델들의 매개변수의 수와 비교할수 있게 해보자

In [85]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


## Train the Model

우선 옵티마이저를 생성한다. 여기에는 모듈의 매개변수들을 업데이트 할 수 있는 알고리즘이 있다. 여기서, 우리는 stochastic gradient descent를 이용한다. 

In [86]:
import torch.optim as optim
optimizer = optim.SGD(model.parameters(), lr=1e-3)

이제 손실 함수를 살펴보자. 파이토치에서는 이걸 표준으로 부른다.
여기서 손실 함수는 binary cross entropy with logits을 사용한다.

사용한 모델은 현재 고정되지않은 숫자를 아웃풋으로 낸다. 우리의 레이블은 0 또는 1이여야 하기 때문에 우리는 예측값을 0에서 1사이의 숫자로 한정해야 한다. 따라서 시그모이드 또는 로지스틱 함수를 사용하기로 한다.

그리고 이진 교차 엔트로피를 사용하여 손실을 계산하기 위해 bound scalar을 사용한다.

BCEWithLogitsLoss criterion은 시그모이드와 이진 교차엔트로피 방법을 모두 수행한다.

In [87]:
criterion = nn.BCEWithLogitsLoss()

.to를 사용함으로써 모델과 criterion을 GPU에 실을 수 있다.

In [88]:
model = model.to(device)
criterion = criterion.to(device)

우리의 criterion 함수는 손실을 계산하지만 정확도를 계산하기위해 함수를 만들 필요가 있다.
이 함수는 처음에 예측값들을 시그모이드 층을 통하여 통과시키고 값을 0과 1 사이로 압축시키고
그걸 근접한 정수에 배치시킨다. 0.5에서 1 사이의 값은 긍정으로 분류되고 나머지는 부정이다.

그다음 얼마나 근사된 예측치가 실제 레이블과 동일한지 계산할 수 있고 배치를 통하여 평균을
낼 수 있다.

In [89]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this
    returns 0.8, NOT8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

train 함수는 한 배치에 한번씩 돌면서 모든 샘플들을 반복한다.
model.train()은 모델을 트레이닝 모드에 놓는데 사용되며 드롭아웃과 배치 정규화를 사용한다. 비록 이 모델에서 이들을 사용하진 않지만, 포함시켜보는것도 좋은 연습이 된다.

각각의 배치마다 우리는 처음에 경사를 0으로만든다. 한 모델 안의 각각의 매개변수들은 criterion에 의해서 계산된 경사값을 grad 특성에 가지고 있다. 파이토치는 이 경사들을 마지막 경사 계산을 통해 자동적으로 제거하거나 0으로만들지 않으므로 수동으로 0으로 만들어야 한다.

그다음으로 문장들의 배치인 batch.text를 모델에 실어야한다. 그렇다고 model.forward(batch.text)를 할 필요는 없다. 예측값들이 초기에 [batch size,1]이고 차원의 크기를 1로 줄여야하기 때문에 squeeze가 필요하다.

손실과 정확도는 예측값과 레이블들,batch.label을 사용하여 손실이 모든 샘플들의 배치를 돌면서 평균낸 값으로 계산된다. 각각의 매개변수들의 경사값은 loss.backward()를 통하여 계산되고 optimizer.step()의 옵티마이저 알고리즘을 통하여 업데이트된다.

손실과 정확성은 에폭을 돌면서 축적되고 .item()이 하나의 값을 가지고 있는 텐서로부터 스칼라 값을 추출하기 위해 사용된다.

마지막으로, 에폭을돌면서 평균된 손실과 정확도값을 반환한다. 이터레이터의 길이는 이터레이터 안의 배치들의 수와 같다.

LABEL field를 초기화할 때 우리는 토치값을 실수로 고정한다. 이는 왜냐하면 우리 criterion은 input값을 FloatTensor로 여기는 반면 TorchText는 LongTensor를 디폴트 값으로 가지기 때문이다. 대안은 criterion에 batch.label을 쓰는 대신에 train 함수를 batch.label.float()에 통과시켜 변환하는 것이다.

In [93]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

model.eval()는 모델을 evaluation mode에 넣고 드롭아웃과 배치 정규화를 사용한다.
또다시 이 모델에서 사용하진 않지만 좋은 연습이도리거다
With no_grad()에서는 파이토치 연산에서 경사값 계산이 되지 않으므로 메모리 소모가 적어
연산 속도를 늘릴 수 있다.
나머지 함수는 train함수와 같다. optimizer.zero_grad(),loss.backward(), 
optimizer.step()을 제거함으로써 평가도중 모델의 파라미터들을 제거하진 않는다.

In [96]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

또한 모델들 간의 트레이닝 시간을 비교하기 위해서 한 에폭을 도는데 시간이 얼마나 걸리는지
측정하는 함수를 생성한다.

In [91]:
import time

def epoch_time(start_time, end_time_):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

이제 멀티플 에폭을 돌면서 모델을 트레인시키고, 에폭은 training , validation sets를 모두 완전히 통과한다.

각각의 에폭에서, 만약 validation loss가 지금까지 본 것중에 가장 적다면 , 그 모델의 파라미터들을 저장해서 트레이닝이 끝난 후에 테스트 셋에다 그 모델을 적용한다.

In [97]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 47s
	Train Loss: 0.693 | Train Acc: 50.11%
	 Val. Loss: 0.696 |  Val. Acc: 48.95%
Epoch: 02 | Epoch Time: 0m 48s
	Train Loss: 0.693 | Train Acc: 49.87%
	 Val. Loss: 0.695 |  Val. Acc: 50.46%
Epoch: 03 | Epoch Time: 0m 48s
	Train Loss: 0.693 | Train Acc: 49.28%
	 Val. Loss: 0.696 |  Val. Acc: 48.97%
Epoch: 04 | Epoch Time: 0m 48s
	Train Loss: 0.693 | Train Acc: 50.17%
	 Val. Loss: 0.696 |  Val. Acc: 48.97%
Epoch: 05 | Epoch Time: 0m 48s
	Train Loss: 0.693 | Train Acc: 50.14%
	 Val. Loss: 0.695 |  Val. Acc: 48.90%


위 결과를 보고 손실이 실제로 잘 줄어들지 않았고 정확도가 떨어진다는 것을 알 수 있다.
이것은 모델과 관련된 몇몇 문제가 있고 이를 다음 노트에서 개선할 것이다.

마지막으로 테스트 셋에대한 손실과 정확도를 볼 것인데 이는 가장 좋은 validation loss 로 부터 얻어낸 매개변수들을 사용할 것이다.

In [98]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f}) |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.677) |  Test Acc: 61.85%


## Nexp steps

다음 notebook에서 정확성을 향상시키기 위해 다룰것들은 다음과 같다

packed padded sequences
pre-trained word embeddings
different RNN architecture
bidirectional RNN
multi-layer RNN
regularization
a different optimizer

이들은 ~84% 의 정확성 향상까지 도와줄 것이다.